In [1]:
import pandas as pd
import os
import numpy as np

import Utils as ut
import CortesAlignmentFile as ca
import mySampler as ms
 
from sklearn.model_selection import StratifiedShuffleSplit, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.preprocessing import normalize

## Dataset Configuration

In [2]:
d_toy = pd.read_csv(os.path.join('data', 'toyDataset.csv'))
outputs = pd.read_csv(os.path.join('data', 'toyLabel.csv'))

In [3]:
toy = d_toy.values
y = outputs.values.reshape(-1)

In [4]:
# COMPUTATIONAL COMPLEXITY: Reduce #samples
tr_idx, ts_idx = next(StratifiedShuffleSplit(n_splits=1, test_size=0.25).split(toy, y))

In [5]:
ds1 = np.hstack([toy[:,:5], toy[:,10:15]])
ds1_tr = ds1[tr_idx]
ds1_ts = ds1[ts_idx]

ds2 = np.hstack([toy[:,5:10], toy[:,15:20]])
ds2_tr = ds2[tr_idx]
ds2_ts = ds2[ts_idx]

ds3 = toy[:,20:]
ds3_tr = ds3[tr_idx]
ds3_ts = ds3[ts_idx]

y_ = y[tr_idx]
y_test = y[ts_idx]

toy_ = ds1_tr
toy_test = ds1_ts

## Utilities

In [6]:
def centering_normalizing(train, test, exclusion_list = None):
    
    if exclusion_list is not None:
        scale, train = ut.centering_normalizing(train, exclusion_list)
    else:
        scale, train = ut.centering_normalizing(train)

    new_Xts = test-scale

    if exclusion_list is not None:
        new_Xts[:, exclusion_list] = test[:, exclusion_list]

    return train, new_Xts, scale


def normalizing(train, test):
    return normalize(train), normalize(test)


def make_regression(X, y, sparsity):

    scores = np.zeros(len(sparsity))
    
    for idx, sp in enumerate(sparsity):
        model = LogisticRegression(penalty = 'l1', C = sp)
        scores[idx] = np.mean(cross_validate(model,  X, y, return_train_score=False, cv=3)['test_score'])
        
    return sparsity[np.argmax(scores)]


def learn(C_, C_test, y, y_test, sparsity, centering = False, norm = False):
    
    if centering == True:
        C_, C_test, _ = centering_normalizing(C_, C_test)#, [5,6,7,9,10,13,15,16,17,18,19])

    if norm == True:
         C_, C_test = normalizing(C_, C_test)

    best_alpha = make_regression(C_, y, sparsity)
    model = LogisticRegression(penalty = 'l1', C = best_alpha)
    model.fit(C_, y)
    y_pred = model.predict(C_test)
    coef = model.coef_
    
    accuracy = ut.balanced_accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    print("\tAccuracy: {}".format(accuracy))
    print("\tPrecision: {}".format(precision))
    print("\tRecall: {}".format(recall))
    print("Coef: {}".format(coef))
    print("BestLambda: {}".format(best_alpha))

In [7]:
sparsity = list(np.arange(0.1, 2, 0.2))

## Origin Data Centering

### Cardio

In [11]:
learn(toy_, toy_test, y_, y_test, sparsity, centering = True)

	Accuracy: 0.5472972972972974
	Precision: 0.5365853658536586
	Recall: 0.5945945945945946
Coef: [[0.         0.         0.         0.         0.         0.
  0.70943101 0.         0.         0.        ]]
BestLambda: 0.1


## Origin Data  Centering and Normalization

### Cardio

In [13]:
learn(toy_, toy_test, y_, y_test, sparsity, centering = True, norm = True)

	Accuracy: 0.5472972972972974
	Precision: 0.5365853658536586
	Recall: 0.5945945945945946
Coef: [[0.         0.         0.         0.         0.         0.
  0.70943101 0.         0.         0.        ]]
BestLambda: 0.1
